**This Jupyter Notebook requires a Julia 1.10 Kernel**

In [1]:
using Pkg

In [2]:
Pkg.activate(".")

  Activating project at `~/Dokumente/Projects/CPS/ABZ/abz2025-case-study/versaille/analysis`


In [3]:
try
    # Python Initialization may fail due to insufficient Python environment, but this is irrelevant for this Notebook
    using SNNT
catch
end

In [4]:
using SNNT
using JLD
using Glob
using Plots
using PyCall

In [5]:
function load_stars(folder, prefix, limit)
    println("Loading results from $folder/$prefix-*.jld")
    stars = []
    for file in glob("$folder/$prefix-*.jld")
        println("Loading $file")
        res = load(file)
        for res in res["result"]
            append!(stars, res.stars)
        end
        if length(stars) > limit
            break
        end
    end
    return stars
end

load_stars (generic function with 1 method)

In [6]:
function summarize_and_load(folder, prefix)
    println("Loading results from $folder/$prefix-*.jld")
    results = []
    metadata = nothing
    for file in glob("$prefix-*.jld",folder)
        println(file)
        if occursin("summary",file)
            continue
        end
        cur_results = load(file)
        if haskey(cur_results,"backup_meta")
            metadata = cur_results["backup_meta"]
        end
        append!(results,cur_results["result"])
    end
    result_summary = SNNT.VerifierInterface.reduce_results(results)
    save("$folder/$prefix-summary.jld","result",result_summary,"args",metadata)
    return (result_summary, metadata)
end

summarize_and_load (generic function with 1 method)

In [7]:
results = load_stars("../results", "iter-2-cars-full",20000)
nothing

Loading results from ../results/iter-2-cars-full-*.jld
Loading ../results/iter-2-cars-full-17.jld
Loading ../results/iter-2-cars-full-19.jld
Loading ../results/iter-2-cars-full-2.jld
Loading ../results/iter-2-cars-full-24.jld
Loading ../results/iter-2-cars-full-25.jld
Loading ../results/iter-2-cars-full-28.jld
Loading ../results/iter-2-cars-full-3.jld
Loading ../results/iter-2-cars-full-34.jld
Loading ../results/iter-2-cars-full-37.jld
Loading ../results/iter-2-cars-full-4.jld
Loading ../results/iter-2-cars-full-5.jld
Loading ../results/iter-2-cars-full-6.jld
Loading ../results/iter-2-cars-full-final.jld


In [8]:
println("Total Stars Retrieved: ",length(results))
# Filtering out counterexampels where front car position is too close due to float rounding
# This is a problem with the counterexample extraction from Z3 *not* with the counterexamples themselves
function dl_invariant(cex)
    xf = 5*40*cex[2]
    vf = 2*40*cex[4]
    xl = 5*40*(cex[2]+cex[7])
    vl = 2*40*(cex[4]+cex[9])
    return ((vl >= 0) & (vf >= 0) & (vl <= 40.0) & (vf <= 40.0) & (xf + 5.0 <= xl) & (xf + vf^2/(2*3.0) + 5.0 < xl + vl^2/(2*5.0)))
end

results_float = filter(x -> !(dl_invariant(x.counter_example[1])), results)
results = filter(x -> dl_invariant(x.counter_example[1]), results)
println("Total Stars After Filtering: ",length(results))

Total Stars Retrieved: 14917
Total Stars After Filtering: 14917


In [9]:
maximum([results[i].counter_example[1][7] for i in 1:length(results)])

0.9999999850988388

In [10]:
minimum([results[i].counter_example[1][7] for i in 1:length(results)])

0.025000035762786865

In [11]:
pushfirst!(pyimport("sys")."path", "");

In [12]:
pyimport("gymnasium")

PyObject <module 'gymnasium' from '/home/samuel/anaconda3/envs/abz-env/lib/python3.10/site-packages/gymnasium/__init__.py'>

In [13]:
py"""
from highway_reproduction import *
from custom_highway import *

# Acceleration control bug: Must be >0 to avoid car accidentally going backwards...
BrakingVehicle.MIN_SPEED = 1.0

from highway_env.vehicle.kinematics import Vehicle
from highway_env.vehicle.behavior import IDMVehicle
Vehicle.MIN_SPEED = 0.0
IDMVehicle.ACC_MAX = 5.0
IDMVehicle.MIN_SPEED = 0.0

envs_all = reproduce_crash_get_envs(None)
envs_all = [env for _,env in envs_all]
model = load_model("../nets/highspeed-rew-1.zip")
envs_worst_case = [envs_all[2]]

def get_results(observation):
    global envs_all
    global model
    set_observation_seed(envs_all, observation)
    results = reproduce_crash_evaluate(envs_all, model, test_runs=1, default_action=0)
    return results

def get_results_wc(observation):
    global envs_worst_case
    global model
    set_observation_seed(envs_worst_case, observation)
    results = reproduce_crash_evaluate(envs_worst_case, model, test_runs=1, default_action=0)
    return results
"""
get_results = py"get_results"
get_results_wc = py"get_results_wc"

/home/samuel/anaconda3/envs/abz-env/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
/home/samuel/anaconda3/envs/abz-env/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
/home/samuel/anaconda3/envs/abz-env/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flatten

PyObject <function get_results_wc at 0x7444e3923b50>

In [14]:
length(results)

14917

In [15]:
crash_results = []
@time for star in results[1:100]
    push!(crash_results,get_results(star.counter_example[1]))
end

 94.028041 seconds (331.43 k allocations: 22.084 MiB, 0.31% compilation time)


In [17]:
function summarize_crashes(crash_results)
    default_env_nn = 0
    default_env_nn_rew = []
    low_brake_env_nn = 0
    low_brake_env_nn_rew = []
    front_brake_env_nn = 0
    front_brake_env_nn_rew = []
    default_env_brake = 0
    default_env_brake_rew = []
    low_brake_env_brake = 0
    low_brake_env_brake_rew = []
    front_brake_env_brake = 0
    front_brake_env_brake_rew = []
    for res in crash_results
        if res[1][1][1] == 1
            default_env_nn += 1
        end
        if res[1][1][2] == 1
            low_brake_env_nn += 1
        end
        if res[1][1][3] == 1
            front_brake_env_nn += 1
        end
        if res[2][1][1] == 1
            default_env_brake += 1
        end
        if res[2][1][2] == 1
            low_brake_env_brake += 1
        end
        if res[2][1][3] == 1
            front_brake_env_brake += 1
        end
        push!(default_env_nn_rew,res[1][2][1])
        push!(low_brake_env_nn_rew,res[1][2][2])
        push!(front_brake_env_nn_rew,res[1][2][3])
        push!(default_env_brake_rew,res[2][2][1])
        push!(low_brake_env_brake_rew,res[2][2][2])
        push!(front_brake_env_brake_rew,res[2][2][3])
    end
    println("Crashes:")
    println("Default Env NN:\t\t\t",default_env_nn)
    println("Low Brake Env NN:\t\t",low_brake_env_nn)
    println("Front Brake Env NN:\t\t",front_brake_env_nn)
    println("Default Env Brake:\t\t",default_env_brake)
    println("Low Brake Env Brake:\t\t",low_brake_env_brake)
    println("Front Brake Env Brake:\t\t",front_brake_env_brake)
    return (default_env_nn_rew, low_brake_env_nn_rew, front_brake_env_nn_rew, default_env_brake_rew, low_brake_env_brake_rew, front_brake_env_brake_rew)
end

summarize_crashes (generic function with 1 method)

In [18]:
summarize_crashes(crash_results)

Crashes:
Default Env NN:			0
Low Brake Env NN:		0
Front Brake Env NN:		100
Default Env Brake:		0
Low Brake Env Brake:		0
Front Brake Env Brake:		0


(Any[16.25, 16.09375, 15.9375, 15.9375, 16.09375, 15.9375, 15.9375, 16.09375, 15.9375, 16.09375  …  15.9375, 15.9375, 16.2158203125, 16.09375, 16.2109375, 16.09375, 16.09375, 15.9375, 16.09375, 15.9375], Any[17.099999999999984, 16.865624999999998, 17.099999999999984, 17.099999999999984, 16.865624999999998, 17.099999999999984, 17.099999999999984, 16.865624999999998, 17.099999999999984, 16.865624999999998  …  16.662499999999998, 16.662499999999998, 17.072460937499997, 16.865624999999998, 17.057812499999997, 16.865624999999998, 16.865624999999998, 16.662499999999998, 16.865624999999998, 16.662499999999998], Any[4.3688199999999995, 4.481849999999999, 3.793499999999999, 3.793499999999999, 4.481849999999999, 3.750987999999999, 3.763019999999999, 4.470569999999999, 3.777099999999999, 4.470569999999999  …  1.7490999999999999, 1.8249999999999997, 4.562904687499999, 3.8841249999999987, 4.535890499999999, 2.5874999999999995, 2.5194749999999995, 1.7490999999999999, 2.5503749999999994, 1.7206199999

In [28]:
crash_results = []
@time for star in results
    push!(crash_results,get_results(star.counter_example[1]))
end

14075.025968 seconds (2.38 M allocations: 87.992 MiB, 0.00% gc time)


In [29]:
save("2-cars-crash_results.jld","crash_results",crash_results)

# Analysis of Crashes

In [16]:
crash_results = load("2-cars-crash_results.jld")["crash_results"]

14917-element Vector{Any}:
 (([0, 0, 1], [16.25, 17.099999999999984, 4.3688199999999995]), ([0, 0, 0], [15.375, 15.712499999999999, 15.712499999999999]))
 (([0, 0, 1], [16.09375, 16.865624999999998, 4.481849999999999]), ([0, 0, 0], [15.4375, 15.809375, 15.809375]))
 (([0, 0, 1], [15.9375, 17.099999999999984, 3.793499999999999]), ([0, 0, 0], [15.375, 15.712499999999999, 15.712499999999999]))
 (([0, 0, 1], [15.9375, 17.099999999999984, 3.793499999999999]), ([0, 0, 0], [15.375, 15.712499999999999, 15.712499999999999]))
 (([0, 0, 1], [16.09375, 16.865624999999998, 4.481849999999999]), ([0, 0, 0], [15.4375, 15.809375, 15.809375]))
 (([0, 0, 1], [15.9375, 17.099999999999984, 3.750987999999999]), ([0, 0, 0], [15.375, 15.712499999999999, 15.712499999999999]))
 (([0, 0, 1], [15.9375, 17.099999999999984, 3.763019999999999]), ([0, 0, 0], [15.375, 15.712499999999999, 15.712499999999999]))
 (([0, 0, 1], [16.09375, 16.865624999999998, 4.470569999999999]), ([0, 0, 0], [15.4375, 15.809375, 15.809375])

In [19]:
summarize_crashes(crash_results)

Crashes:
Default Env NN:			538
Low Brake Env NN:		617
Front Brake Env NN:		3593
Default Env Brake:		0
Low Brake Env Brake:		0
Front Brake Env Brake:		0


(Any[16.25, 16.09375, 15.9375, 15.9375, 16.09375, 15.9375, 15.9375, 16.09375, 15.9375, 16.09375  …  17.117421874999994, 17.117421874999994, 17.1396875, 17.141562500000003, 17.071562500000002, 17.252187500000005, 17.1396875, 17.1396875, 16.826484375, 16.887578125], Any[17.099999999999984, 16.865624999999998, 17.099999999999984, 17.099999999999984, 16.865624999999998, 17.099999999999984, 17.099999999999984, 16.865624999999998, 17.099999999999984, 16.865624999999998  …  17.298828124999996, 17.298828124999996, 17.320312499999996, 17.320312499999996, 17.363281249999996, 17.320312499999996, 17.320312499999996, 17.320312499999996, 17.298828124999996, 17.298828124999996], Any[4.3688199999999995, 4.481849999999999, 3.793499999999999, 3.793499999999999, 4.481849999999999, 3.750987999999999, 3.763019999999999, 4.470569999999999, 3.777099999999999, 4.470569999999999  …  17.298828124999996, 17.298828124999996, 17.320312499999996, 17.320312499999996, 17.363281249999996, 17.320312499999996, 17.320312

In [25]:
default_env_crash = map(x -> x[1][1][1] > 0, crash_results)
low_brake_env_crash = map(x -> x[1][1][2] > 0, crash_results)
front_brake_env_crash = map(x -> x[1][1][3] > 0, crash_results)
print(sum(default_env_crash)," ",sum(low_brake_env_crash)," ",sum(front_brake_env_crash))

538 617 3593

In [26]:
interesting_results = results[default_env_crash]

538-element Vector{Any}:
 SNNT.VerifierInterface.Star(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.011408593 0.20840992 … -0.050983407 0.014868651; 0.009161299 -0.09131165 … -0.25063127 0.005963117], Float32[-0.0, 0.35355338, 0.01767767, 0.01767767, 0.01767767, 0.55378324, 0.9121081, 0.5026186, 0.0014403465, 0.93318886  …  0.208888, -0.09046932, 0.06034525, -0.1562662, 0.23050623, 0.10619328, -0.013058278, -0.072583765, -0.02024087, 0.06677414], Float32[-0.0965659 5.3162403 … 1.880786 -0.24354395; 0.23478973 1.2814128 … 0.21566463 -0.0064086225; 0.80460054 0.8195448 … -0.16698694 -0.3450244], Float32[1.215942, 0.58444345, 0.6520623], [(1.0, 1.0), (0.11671157608934254, 0.12277742869690723), (0.0, 0.0), (0.39108310167991206, 0.3942470560656302), (0.0, 0.0), (1.0, 1.0), (0.04074181036117967, 0.044838056898063014), (0.0, 0.0), (0.10552022190467655, 0.10891689832008793), (0.0, 0.0)  …  (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.

In [27]:
argmax(map(x->x.counter_example[1][9],interesting_results))

287

In [30]:
println(interesting_results[287].counter_example[1])

[1.0, 0.0011208171058602599, 0.0, 0.3359375, 0.0, 1.0, 0.0302734375, 0.0, 0.15625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [31]:
get_results(interesting_results[287].counter_example[1])

(([1, 1, 1], [3.118063, 3.118063, 2.268125]), ([0, 0, 0], [15.7421875, 16.3375, 16.3375]))